# Project: Bollinger Bands Mean Reversion Strategy

When prices move far from their average (more than a few standard deviations), they tend to revert.
We’ll go long when the price dips below the lower Bollinger Band (oversold) and short when it rises above the upper band (overbought).

We’ll only take Bollinger Band mean reversion signals when the RSI confirms oversold or overbought conditions — improving signal quality and avoiding “catching falling knives.”

## Imports

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

%load_ext autoreload
%autoreload 2

## Data Setup

In [ ]:
ticker = 'AAPL'
start_date = '2015-01-01'
end_date = datetime.today().strftime('%Y-%m-%d')

ticker, start_date, end_date

In [ ]:
df = yf.download(ticker, start=start_date, end=end_date)
df = df.droplevel(axis=1, level=1)['Close'].to_frame()
df.dropna(inplace=True)
df.head()

## Set Indicators

In [ ]:
window = 20
num_std = 2
rsi_period = 14

In [ ]:
df

In [ ]:
df['MA'] = df['Close'].rolling(window).mean()
df['STD'] = df['Close'].rolling(window).std()
df['Upper'] = df['MA'] + num_std * df['STD']
df['Lower'] = df['MA'] - num_std * df['STD']

In [ ]:
df

In [ ]:
df['delta'] = df['Close'].diff()

In [ ]:
df

In [ ]:
# RSI Calculation

delta = df.Close.diff()
gain = np.where(delta > 0, delta, 0)
loss = np.where(delta < 0, -delta, 0)
avg_gain = pd.Series(gain).rolling(rsi_period).mean()
avg_loss = pd.Series(loss).rolling(rsi_period).mean()
rs = (avg_gain / avg_loss).values
df['RSI'] = 100 - (100 / (1 + rs))

df.dropna(inplace=True)
df


## Generate Trading Signals

Conditions:
* Long when price < lower band AND RSI < 30 (oversold)
* Short when price > upper band AND RSI > 70 (overbought)
* Exit when price crosses back towards MA

In [ ]:
position = 0 # 1 = LONG, -1 = SHORT, 0 = FLAT
df['Position'] = position # Initial FLAT (No position)

for i in range(1, len(df)):

    price = df['Close'].iloc[i] # Curent Price
    prev_price = df['Close'].iloc[i-1] # Previous Price
    ma = df['MA'].iloc[i]
    prev_ma = df['MA'].iloc[i-1]
    rsi = df['RSI'].iloc[i]
    upper = df['Upper'].iloc[i]
    lower = df['Lower'].iloc[i]

    # ENTRY RULES
    if position == 0: # FLAT
        if price < lower and rsi < 30: # If price crosses the lower band & stock is undersold
            position = 1 # Open a LONG position
        elif price > upper and rsi > 70: # If price crosses the upper band & stock is oversold
            position = -1 # Open a SHORT position

    # EXIT RULES
    elif position == 1 and price < ma and prev_price > prev_ma:
        position = 0 # Exit LONG when price crosses below MA
    elif position == -1 and price > ma and prev_price < prev_ma:
        position = 0 # Exit SHORT when price crosses above MA

    df.loc[df.index[i], 'Position'] = position

df

    

In [ ]:
df['Position'].value_counts()

## Backtesting

In [ ]:
df['Return'] = df['Close'].pct_change()
df['Strategy'] = df['Position'] * df['Return']

df['Strategy_Curve'] = (1 + df['Strategy']).cumprod()
df['BuyHold_Curve'] = (1 + df['Return']).cumprod()

df

## Performance Evaluation

In [ ]:
def performance_metrics(df):
    strategy = df['Strategy']
    total_return = df['Strategy_Curve'].iloc[-1] - 1
    cagr = (df["Strategy_Curve"].iloc[-1]) ** (252 / len(df)) - 1
    sharpe = np.sqrt(252) * strategy.mean() / strategy.std()
    max_dd = (df["Strategy_Curve"].cummax() - df["Strategy_Curve"]).max()
    
    return {
        "Total Return": f"{total_return:.2%}",
        "CAGR": f"{cagr:.2%}",
        "Sharpe Ratio": round(sharpe, 2),
        "Max Drawdown": f"{max_dd:.2%}"
    }

In [ ]:
performance_metrics(df)

## Visualization

### Bollinger Bands Plot With Long/Short Signals

In [ ]:
def plot_bands(df):
    fig, ax = plt.subplots(figsize=(16,8))
    ax.set_title('Bollinger Bands', fontsize=16)
    ax.plot(df['Close'], label = 'Close', linewidth=1)
    ax.plot(df['MA'], label = 'MA', linewidth=1, alpha=0.6)
    ax.plot(df['Upper'], label = 'Upper Band', linestyle='--', color='grey', linewidth=1)
    ax.plot(df['Lower'], label = 'Lower Band', linestyle='--', color='grey', linewidth=1)
    ax.fill_between(df.index, df['Upper'], df['Lower'], color='grey', alpha=0.075)


    ax.scatter(x=df[df['Signal']==1].index, y=df[df['Signal']==1]['Close'].loc[df[df['Signal']==1].index], marker='^', s=30, color='Green', label='Long Signal', zorder=4, alpha=0.6)
    ax.scatter(x=df[df['Signal']==-1].index, y=df[df['Signal']==-1]['Close'].loc[df[df['Signal']==-1].index], marker='v', s=30, color='Red', label='Short Signal', zorder=4, alpha=0.6)

    ax.grid(True, alpha=0.4)
    ax.spines[['top', 'right']].set_visible(False)
    ax.legend()

    plt.show();

    return

In [ ]:
plot_bands(df.loc['2023':])

### RSI Plot

In [ ]:
def plot_rsi(df):
    fig, ax = plt.subplots(figsize=(16,8))
    ax.set_title('RSI', fontsize=16)
    ax.plot(df['RSI'], label = 'RSI', linewidth=1, color='y')
    ax.axhline(70, color="r", linestyle="--", alpha=0.7)
    ax.axhline(30, color="g", linestyle="--", alpha=0.7)
    ax.fill_between(df.index, 70, 100, color='r', alpha=0.075, label='Overbought')
    ax.fill_between(df.index, 0, 30, color='g', alpha=0.075, label='Oversold')

    ax.grid(True, alpha=0.6)

    ax.spines[['top', 'right', 'left']].set_visible(False)
    ax.legend()
    
    ax.set_yticks([30,70])
    ax.tick_params(axis='y', labelsize=12, length=0)
    plt.show();

    return

plot_rsi(df)


### Strategy vs Buy&Hold

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(df["Strategy_Curve"], label="Strategy")
plt.plot(df["BuyHold_Curve"], label="Buy & Hold")
plt.title(f"{ticker} Strategy vs Buy & Hold")
plt.legend()
plt.grid(True)
plt.show()